In [41]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import re
import csv

import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter

import nltk
from textblob import TextBlob
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('vader_lexicon')
from nltk.corpus import conll2000
nltk.download('conll2000')
nltk.download('stopwords')

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize.toktok import ToktokTokenizer

!pip install spacy
!python -m spacy download es_core_news_sm
!pip install --upgrade spacy
import spacy

# Cargar el modelo de lenguaje en español
nlp = spacy.load("es_core_news_sm")

#para la ortografia
import enchant

#para la corrección de la fecha
from datetime import datetime

#para los acentos
!pip install unidecode
from unidecode import unidecode



[nltk_data] Downloading package punkt to /Users/sil/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/sil/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package conll2000 to /Users/sil/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/sil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 24.3 MB/s eta 0:00:0000:0100:01


✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


# sacamos los datos

In [107]:
# Ruta al archivo de texto que deseas convertir
archivo_txt = '../data/capi_chat.txt'

# Ruta al archivo CSV de salida
archivo_csv = '../data/capi_chat.csv'

# Patrón de expresión regular para buscar la fecha, hora, persona y mensaje
patron = r'\[(.*?)\] (.*?): (.*)'

# Lee el archivo de texto y escribe los datos en un archivo CSV
with open(archivo_txt, 'r') as txt_file, open(archivo_csv, 'w', newline='') as csv_file:
    # Crea un objeto de escritura de CSV
    writer = csv.writer(csv_file)
    # Escribe el encabezado del CSV
    writer.writerow(["Fecha", "Hora", "Persona", "Mensaje"])
    
    # Procesa cada línea del archivo de texto
    for linea in txt_file:
        # Busca coincidencias con el patrón de expresión regular
        match = re.match(patron, linea)
        
        # Si hay una coincidencia, extrae la información
        if match:
            fecha_hora, persona, mensaje = match.groups()
            fecha, hora = fecha_hora.split()
            
            # Escribe los datos en el archivo CSV
            writer.writerow([fecha, hora, persona, mensaje])

print("El archivo TXT se ha convertido exitosamente a CSV.")


El archivo TXT se ha convertido exitosamente a CSV.


In [94]:
df = pd.read_csv("../data/capi_chat.csv")
df

,Fecha,Hora,Persona,Mensaje
0,"30/11/23,",19:12:50,Capibaras Ironhack,‎Los mensajes y las llamadas están cifrados de...
1,"30/11/23,",19:12:50,Alana Ironhack,‎Alana Ironhack creó el grupo “Capibaras Ironh...
2,"30/11/23,",19:12:50,Capibaras Ironhack,‎Alana Ironhack te añadió.
3,"30/11/23,",19:12:56,Alana Ironhack,Hola
4,"30/11/23,",19:13:12,Alana Ironhack,Les paso las fotos de ayer prepárense xq es un...
...,...,...,...,...
1769,"17/2/24,",21:09:53,Sil🐨,No y que también pensaba que esos días que íba...
1770,"17/2/24,",21:11:04,Alana Ironhack,Jajaj
1771,"17/2/24,",21:11:18,Alana Ironhack,Disfrutar de tu libertad
1772,"17/2/24,",21:18:17,Sil🐨,JAJAJAJAJA


# Limpieza

In [95]:
# Función para eliminar caracteres especiales de un texto

def eliminar_caracteres_especiales(texto):
    if isinstance(texto, str):
        # Utiliza una expresión regular para eliminar todos los caracteres que no sean letras o números
        texto_limpio = re.sub(r'[^a-zA-Z0-9\s]', '', texto)
        return texto_limpio
    else:
        return texto

# Aplicar la función de eliminación de caracteres especiales a cada elemento de la Serie 'Mensaje'
df['Mensaje'] = df['Mensaje'].apply(eliminar_caracteres_especiales)

# Mostrar los resultados
print("DataFrame con mensajes corregidos y limpiados:")
df.head(15)

DataFrame con mensajes corregidos y limpiados:


,Fecha,Hora,Persona,Mensaje
0,"30/11/23,",19:12:50,Capibaras Ironhack,Los mensajes y las llamadas estn cifrados de e...
1,"30/11/23,",19:12:50,Alana Ironhack,Alana Ironhack cre el grupo Capibaras Ironhack
2,"30/11/23,",19:12:50,Capibaras Ironhack,Alana Ironhack te aadi
3,"30/11/23,",19:12:56,Alana Ironhack,Hola
4,"30/11/23,",19:13:12,Alana Ironhack,Les paso las fotos de ayer preprense xq es un ...
5,"30/11/23,",20:22:19,~ Teresa,Es un bombardeo que nos ahorra prcticamente un...
6,"30/11/23,",20:22:38,Alana Ironhack,Jajaja
7,"30/11/23,",20:32:18,Sil🐨,Muchas gracias
8,"1/12/23,",17:10:16,~ Teresa,Fan del icono del grupo
9,"1/12/23,",17:11:26,~ Teresa,Entregue el lab sin el ltimo ejercicio del fac...


In [96]:
#En la colum fecha se queda una ',' vamos a quitarla

In [97]:
fecha = df['Fecha']

# Eliminar la coma de las fechas
fechas_sin_coma = fecha.str.replace(",", "")

# Reemplazar la columna 'fecha' en el DataFrame original
df['Fecha'] = fechas_sin_coma
df

,Fecha,Hora,Persona,Mensaje
0,30/11/23,19:12:50,Capibaras Ironhack,Los mensajes y las llamadas estn cifrados de e...
1,30/11/23,19:12:50,Alana Ironhack,Alana Ironhack cre el grupo Capibaras Ironhack
2,30/11/23,19:12:50,Capibaras Ironhack,Alana Ironhack te aadi
3,30/11/23,19:12:56,Alana Ironhack,Hola
4,30/11/23,19:13:12,Alana Ironhack,Les paso las fotos de ayer preprense xq es un ...
...,...,...,...,...
1769,17/2/24,21:09:53,Sil🐨,No y que tambin pensaba que esos das que bamos...
1770,17/2/24,21:11:04,Alana Ironhack,Jajaj
1771,17/2/24,21:11:18,Alana Ironhack,Disfrutar de tu libertad
1772,17/2/24,21:18:17,Sil🐨,JAJAJAJAJA


In [98]:
# ocultamos los nombres

In [99]:
miembros = df['Persona'].unique()
miembros.shape

(4,)

In [100]:
for i in enumerate(miembros):
    df['Persona'] = df['Persona'].replace(i[1], f'persona {i[0]}')

df

,Fecha,Hora,Persona,Mensaje
0,30/11/23,19:12:50,persona 0,Los mensajes y las llamadas estn cifrados de e...
1,30/11/23,19:12:50,persona 1,Alana Ironhack cre el grupo Capibaras Ironhack
2,30/11/23,19:12:50,persona 0,Alana Ironhack te aadi
3,30/11/23,19:12:56,persona 1,Hola
4,30/11/23,19:13:12,persona 1,Les paso las fotos de ayer preprense xq es un ...
...,...,...,...,...
1769,17/2/24,21:09:53,persona 3,No y que tambin pensaba que esos das que bamos...
1770,17/2/24,21:11:04,persona 1,Jajaj
1771,17/2/24,21:11:18,persona 1,Disfrutar de tu libertad
1772,17/2/24,21:18:17,persona 3,JAJAJAJAJA


# Eliminamos acentos

In [101]:
# Mi Serie de Pandas del df, nos quedamos con mensaje 
texto = df['Mensaje']

# Limpiamos el texto
textos_sin_acentos = texto.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

# Ahora textos_sin_acentos contendrá los textos con los acentos eliminados
# lo metemos en df
df['Mensaje'] = textos_sin_acentos
df

,Fecha,Hora,Persona,Mensaje
0,30/11/23,19:12:50,persona 0,Los mensajes y las llamadas estn cifrados de e...
1,30/11/23,19:12:50,persona 1,Alana Ironhack cre el grupo Capibaras Ironhack
2,30/11/23,19:12:50,persona 0,Alana Ironhack te aadi
3,30/11/23,19:12:56,persona 1,Hola
4,30/11/23,19:13:12,persona 1,Les paso las fotos de ayer preprense xq es un ...
...,...,...,...,...
1769,17/2/24,21:09:53,persona 3,No y que tambin pensaba que esos das que bamos...
1770,17/2/24,21:11:04,persona 1,Jajaj
1771,17/2/24,21:11:18,persona 1,Disfrutar de tu libertad
1772,17/2/24,21:18:17,persona 3,JAJAJAJAJA


# cambiamos abreviaturas

In [127]:
# abreviaturas
mapeo_abreviaturas = {
"xq": "porque",
"Holiiii": "Hola",
"sticker omitido": "imagen",
"imagen omitida": "imagen",
"q": "que",
"X": "por",
"x": "por",
"esq": "es que",
"GIF omitido": "GIF",
"Valee": "vale",
"perfe": "perfecto",
"Siiii": "Si",
"siii": "Si",
"Ayyy":"Ay",
"proque": "porque",
"nose": "no se",
"sii": "si",
"tmbn": "tambien",
"lab": "laboratorio",
"labs": "labboratotios",
"lst": "lista",
"reu": "reunion",
"michi": "gato",
"jaajaaj": "jajajajaja",
"xd": "risa",
"capiiis": "capi",
"smpr": "siempre",
"Noup": "No",
"okii": "vale",
"year": "año",
"string": "letras",
"colorinchis": "colores",
"pull request": "subir",
"LOL": "risa a carcajadas",
"Wsp": "WhatsApp",
"Wasap": "WhatsApp",
"wa": "WhatsApp",
"Guasap": "WhatsApp",
"Pq": "porque",
"Xq": "porque",
"q": "que",
"Dnd": "donde",
"K": "que",
"T": "te",
"+/-": "mas o menos",
"Nd": "nada",
"Vdd": "verdad",
"Ntp": "no te preocupes",
"Tqm": "Te quiero mucho",
"Tq": "Te quiero",
"Tmb": "tambien",
"Dps": "despues",
"Xfa": "por favor",
"Dd": "donde",
"Oki": "vale",
"Ok": "vale",
"OK": "vale",
"Tkt": "tranquilo",
"Tqg": "te quiero un monton",
"Bss": "besos",
"Tmbn": "tambien",
"Fvr": "favor",
"Msj": "mensaje",
"LMAO": "risa",
"BRB": "Vuelvo enseguida",
"IDK": "no se",
"idk": "no se",
"ASAP": "tan pronto como sea posible",
"OMG": "Oh Dios mio",
"BFF": "Mejores amigos para siempre",
"Okey": "vale",
}

In [128]:
# Función para corregir abreviaturas en un texto
def corregir_abreviaturas(texto, mapeo_abreviaturas):
    palabras = texto.split()
    texto_corregido = []
    for palabra in palabras:
        palabra_corregida = mapeo_abreviaturas.get(palabra.lower(), palabra)
        texto_corregido.append(palabra_corregida)
    return ' '.join(texto_corregido)

# Llamar a la función corregir_abreviaturas con el diccionario de abreviaturas
resultado = corregir_abreviaturas('T q tal OMG dnd', mapeo_abreviaturas)
print(resultado)

# Mi Serie de Pandas del df, nos quedamos con mensaje 
#texto = df['Mensaje']

# Aplicar la función de corrección de abreviaturas a cada elemento de la Serie
#texto_corregida = texto.apply(lambda texto: corregir_abreviaturas(texto, mapeo_abreviaturas))

# Sustituir la columna 'Mensaje' en el DataFrame con la columna corregida
#df['Mensaje'] = texto_corregida

# Mostrar los resultados
#print("DataFrame con mensajes corregidos:")
#df


T que tal OMG dnd


# palabras derivadas

In [106]:
# Importar NLTK
import nltk
from nltk.stem import SnowballStemmer

# Crear una instancia de SnowballStemmer para español
stemmer_es = SnowballStemmer('spanish')

# Definir una función para realizar el stemming en español
def simple_stemmer_es(texto):
    ps = stemmer_es
    texto = ''.join([ps.stem(word) for word in palabra.split()])
    return texto

# Aplicar la función simple_stemmer_es a la columna 'Mensaje' de tu DataFrame
df['Mensaje_stemmizado'] = df['Mensaje'].apply(simple_stemmer_es)

# Mostrar el resultado
print(df['Mensaje_stemmizado'])


'mi sistema seguir fallar . el suyo fallar ayer , el nuestro falla diariamente .'

# Lexema

In [56]:
# Cargar el modelo de lenguaje de spaCy (asegúrate de haber instalado el modelo adecuado previamente)
nlp = spacy.load("es_core_news_sm")

# Función para lematizar texto
def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

# Sustituir la columna 'Mensaje' en el DataFrame con la columna corregida
df['Mensaje'] = lemmatize_text

# Mostrar los resultados
#print("DataFrame con mensajes lematizados:")
df.head(15)

,Fecha,Hora,Persona,Mensaje
0,30/11/23,19:12:50,persona 0,<function lemmatize_text at 0x7fc06128ba30>
1,30/11/23,19:12:50,persona 1,<function lemmatize_text at 0x7fc06128ba30>
2,30/11/23,19:12:50,persona 0,<function lemmatize_text at 0x7fc06128ba30>
3,30/11/23,19:12:56,persona 1,<function lemmatize_text at 0x7fc06128ba30>
4,30/11/23,19:13:12,persona 1,<function lemmatize_text at 0x7fc06128ba30>
5,30/11/23,20:22:19,persona 2,<function lemmatize_text at 0x7fc06128ba30>
6,30/11/23,20:22:38,persona 1,<function lemmatize_text at 0x7fc06128ba30>
7,30/11/23,20:32:18,persona 3,<function lemmatize_text at 0x7fc06128ba30>
8,1/12/23,17:10:16,persona 2,<function lemmatize_text at 0x7fc06128ba30>
9,1/12/23,17:11:26,persona 2,<function lemmatize_text at 0x7fc06128ba30>


# Eliminación de palabras vacias

In [81]:
import nltk
from nltk.corpus import stopwords
stopword_list_es = nltk.corpus.stopwords.words('spanish')
# Descarga los recursos necesarios de NLTK
nltk.download('punkt')
nltk.download('stopwords')

# Define una función para eliminar las palabras vacías de un texto
def eliminar_stopwords(texto):
    palabras = nltk.word_tokenize(texto)
    stopwords_espanol = set(stopwords.words('spanish'))
    palabras_sin_stopwords = [palabra for palabra in palabras if palabra.lower() not in stopwords_espanol]
    return ' '.join(palabras_sin_stopwords)

# Sustituir la columna 'Mensaje' en el DataFrame con la columna corregida
df['Mensaje'] = eliminar_stopwords
df.head()

[nltk_data] Downloading package punkt to /Users/sil/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/sil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Fecha,Hora,Persona,Mensaje
0,30/11/23,19:12:50,persona 0,<function eliminar_stopwords at 0x7fc063d05f30>
1,30/11/23,19:12:50,persona 1,<function eliminar_stopwords at 0x7fc063d05f30>
2,30/11/23,19:12:50,persona 0,<function eliminar_stopwords at 0x7fc063d05f30>
3,30/11/23,19:12:56,persona 1,<function eliminar_stopwords at 0x7fc063d05f30>
4,30/11/23,19:13:12,persona 1,<function eliminar_stopwords at 0x7fc063d05f30>


In [65]:
import re

def normalize_corpus(corpus, eliminar_caracteres_especiales=True, textos_sin_acentos=True, 
                     corregir_abreviaturas=True, simple_stemmer=True, lemmatize_text=True,
                     eliminar_stopwords=True):
    normalized_corpus = []
    for doc in corpus:
        if eliminar_caracteres_especiales:
            doc = eliminar_caracteres_especiales(doc)
        if textos_sin_acentos:
            doc = textos_sin_acentos(doc)
        if corregir_abreviaturas:
            doc = corregir_abreviaturas(doc)
        if simple_stemmer:
            doc = simple_stemmer(doc)
        if lemmatize_text:
            doc = lemmatize_text(doc)
        if eliminar_stopwords:
            doc = eliminar_stopwords(doc)
        normalized_corpus.append(doc)
    return normalized_corpus


# -----

In [37]:
# diccionario con el valor y nombre de los dias
week = {
    6:'Domingo',
    0:'Lunes',
    1:'Martes',
    2:'Miercoles',
    3:'Jueves',
    4:'Viernes',
    5:'Sabado'
}

# Diccionario con el valor y nombre de los meses
month = {
    1:'Ene',
    2:'Feb',
    3:'Mar',
    4:'Abr',
    5:'May',
    6:'Jun',
    7:'Jul',
    8:'Ago',
    9:'Sept',
    10:'Obt',
    11:'Nov',
    12:'Dic'
}


# dar formato de fecha a los dias 
df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True)

# nueva columna con los nombres de los dias
#df['Dia'] = df['Fecha'].dt.strftime('%A') #(esto tambien lo cambia)
df['Dia'] = df['Fecha'].dt.weekday.map(week)

# nueva columna con el numero del dia
df['Num_Day'] = df['Fecha'].dt.day

# nueva columna con el año 
df['Year'] = df['Fecha'].dt.year

# nueva columna con el numero de mes
df['Num_Month'] = df['Fecha'].dt.month

# nueva columna con el nombre de los meses
df['Month'] = df['Fecha'].dt.month.map(month)

# organizar las columnas
df = df[['Fecha', 'Dia', 'Num_Day','Num_Month', 'Month', 'Year', 'Hora', 'Persona', 'Mensaje']]

# cambiar el tipo de dato de la columna Day
df['Dia'] = df['Dia'].astype('category')
#cambiar formato fecha 
df['Fecha'] = df['Fecha'].dt.strftime('%d/%m/%Y')
df

,Fecha,Dia,Num_Day,Num_Month,Month,Year,Hora,Persona,Mensaje
0,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,persona 0,<function eliminar_stopwords at 0x7fc068356950>
1,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,persona 1,<function eliminar_stopwords at 0x7fc068356950>
2,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,persona 0,<function eliminar_stopwords at 0x7fc068356950>
3,30/11/2023,Jueves,30,11,Nov,2023,19:12:56,persona 1,<function eliminar_stopwords at 0x7fc068356950>
4,30/11/2023,Jueves,30,11,Nov,2023,19:13:12,persona 1,<function eliminar_stopwords at 0x7fc068356950>
...,...,...,...,...,...,...,...,...,...
1769,17/02/2024,Sabado,17,2,Feb,2024,21:09:53,persona 3,<function eliminar_stopwords at 0x7fc068356950>
1770,17/02/2024,Sabado,17,2,Feb,2024,21:11:04,persona 1,<function eliminar_stopwords at 0x7fc068356950>
1771,17/02/2024,Sabado,17,2,Feb,2024,21:11:18,persona 1,<function eliminar_stopwords at 0x7fc068356950>
1772,17/02/2024,Sabado,17,2,Feb,2024,21:18:17,persona 3,<function eliminar_stopwords at 0x7fc068356950>


# exportamos los datos limpios

In [38]:
# Exportamos los datos limpios en un archivo csv
df.to_csv('../data/data_capi_chat_clear.csv', header=True, index=False)

In [52]:
# vamos a darle valor a algunas palabras

In [60]:
valores_de_sentimiento = {
    "feliz": 0.7,
    "bien": 0.5,
    "genial": 0.8,
    "triste": -0.7,
    "cabreado": -0.8,
    "mal": -0.5,
    "espero": 0,
    "caer": -0.1,
    "gripe": -0.1,
    "malo": -0.2,
    "vale perfecto": 0.8,
    "jajajajaja": 0.1,
}


# analisis con TextBlob

In [42]:
mensajes = df['Mensaje'].tolist()

# Lista para almacenar los sentimientos de cada mensaje
sentimientos = []

# Iterar sobre cada mensaje y calcular el sentimiento
for texto in mensajes:
    blob = TextBlob(texto)
    sentimiento = blob.sentiment.polarity
    if sentimiento > 0:
        sentimientos.append("positivo")
    elif sentimiento < 0:
        sentimientos.append("negativo")
    else:
        sentimientos.append("neutro")

# Imprimir los mensajes y sus sentimientos correspondientes
for i, mensaje in enumerate(mensajes):
    #print("Mensaje:", mensaje)
    print("Sentimiento:", sentimientos[i])
    print()

sentimientos


TypeError: The `text` argument passed to `__init__(text)` must be a string, not <class 'function'>

# vamos a probar con VADER

In [61]:
# Crea una instancia del analizador de sentimientos de VADER
sia = SentimentIntensityAnalyzer()

# DataFrame de Pandas con una columna 'Mensaje' que contiene los mensajes de WhatsApp
for texto in df['Mensaje']:
    # Asegurarse de que el texto sea una cadena de texto
    if isinstance(texto, str):
        texto_sin_stopwords = eliminar_stopwords(texto)
        sentimientos = sia.polarity_scores(texto_sin_stopwords)
        print("Texto:", texto)
        print("Sentimientos:", sentimientos)
        print()
    else:
        # Si el valor no es una cadena de texto, imprimir un mensaje de advertencia o manejarlo de otra manera
        print("Mensaje no válido:", texto)


Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb

Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb

Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb

Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb520>
Mensaje no válido: <function eliminar_stopwords at 0x7fc07b6eb

In [44]:
# Creamos listas vacías para almacenar los resultados del análisis de sentimientos
compound_scores = []
positive_scores = []
negative_scores = []
neutral_scores = []

for texto in df['Mensaje']:
    sentimientos = sia.polarity_scores(texto)
    compound_scores.append(sentimientos['compound'])
    positive_scores.append(sentimientos['pos'])
    negative_scores.append(sentimientos['neg'])
    neutral_scores.append(sentimientos['neu'])

# Agregamos las listas de puntuaciones al DataFrame
df['Compound Score'] = compound_scores
df['Positive Score'] = positive_scores
df['Negative Score'] = negative_scores
df['Neutral Score'] = neutral_scores

# Imprimimos el DataFrame con los resultados del análisis de sentimientos agregados
print(df)

AttributeError: 'function' object has no attribute 'encode'

In [63]:

# Crear una instancia del analizador de sentimientos de VADER
sia = SentimentIntensityAnalyzer()

# Paso 2: Calcular Puntajes de Sentimiento por Conversación
puntajes_por_conversacion = {}
for nombre_conversacion, conversacion in conversaciones:
    puntajes_conversacion = []
    for texto in conversacion['Mensaje']:
        sentimientos = sia.polarity_scores(texto)
        puntajes_conversacion.append(sentimientos['compound'])
    puntajes_por_conversacion[nombre_conversacion] = sum(puntajes_conversacion)

# Paso 3: Almacenar los Puntajes de Sentimiento por Conversación
# puntajes_por_conversacion ahora contiene los puntajes de sentimiento agregados por cada conversación

In [64]:
umbral_positivo = 0  # Ejemplo de umbral para clasificar como positivo
umbral_negativo = -0.5  # Ejemplo de umbral para clasificar como negativo

conversaciones_clasificadas = {}
for nombre_conversacion, puntaje in puntajes_por_conversacion.items():
    if puntaje > umbral_positivo:
        clasificacion = 'positiva'
    elif puntaje < umbral_negativo:
        clasificacion = 'negativo'
    else:
        clasificacion = 'neutro'
    conversaciones_clasificadas[nombre_conversacion] = clasificacion

# conversaciones_clasificadas ahora contiene la clasificación (positiva, negativa, neutra) de cada conversación
conversaciones_clasificadas

{'': 'neutro',
 '1': 'neutro',
 '1 rodri': 'neutro',
 '1530h est bien si no se tarda nada': 'neutro',
 '3 silvia': 'neutro',
 '34675233007 del workshop el bonus has hecho lo que harias o has puesto lo harias': 'positiva',
 '34675233007 supongo que estas currando pero para cuando puedas te dejo una duda por aqui jajajaja': 'neutro',
 '34675233007 tu como lo instalaste': 'neutro',
 '911 y ya empieza a preparar para irse': 'neutro',
 'A Jon se lo dijo en unas cerves': 'neutro',
 'A capn': 'neutro',
 'A esa seora la pusieron porque le toc': 'neutro',
 'A lo mejor es una nota del pasado': 'neutro',
 'A lo mejor os pregunto porque estoy ms perdida': 'neutro',
 'A lo poco ya vamos casi todos a las hamburguesas': 'neutro',
 'A m creo que solo me darn tres das por ser marzo todava as que lo estoy cuadrando para pillarme de lunes a mircoles y as aprovechar fin de semana': 'neutro',
 'A m en general me gusta ms como explica Vctor': 'neutro',
 'A m lo de teirme me da miedo por si me queda mal': 'n

In [65]:
# Imprimir los puntajes de sentimiento y las clasificaciones de las conversaciones
for nombre_conversacion, puntaje in puntajes_por_conversacion.items():
    clasificacion = conversaciones_clasificadas[nombre_conversacion]
    print("Conversación:", nombre_conversacion)
    print("Puntaje de Sentimiento:", puntaje)
    print("Clasificación:", clasificacion)
    print()


Conversación: 
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: 1
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: 1 rodri
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: 1530h est bien si no se tarda nada
Puntaje de Sentimiento: -0.296
Clasificación: neutro

Conversación: 3 silvia
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: 34675233007 del workshop el bonus has hecho lo que harias o has puesto lo harias
Puntaje de Sentimiento: 0.5423
Clasificación: positiva

Conversación: 34675233007 supongo que estas currando pero para cuando puedas te dejo una duda por aqui jajajaja
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: 34675233007 tu como lo instalaste
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: 911 y ya empieza a preparar para irse
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: A Jon se lo dijo en unas cerves
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conv

Clasificación: neutro

Conversación: De los nuevos
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: De que vas disfrazada capi
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: De todas formas el certificado te lo darn un mes despus
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: De todas formas lo del EDA yo lo vi una vez y creo que tengo apuntes de ello maana los busco y los paso
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: De todas maneras hoy no a explicado muy bien
Puntaje de Sentimiento: -0.296
Clasificación: neutro

Conversación: De todo un poco
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: De vdd Ay seguro hay otra opcin
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: De vdd Pfff
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: De vdd estoy con 0 ganas de ir
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: Decimos de Rodrigo pero la mente de Ca

Clasificación: neutro

Conversación: Habis echo el ltimo laboratorio
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: Habis visto el mensaje de rocio
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: Hacamos arte y no lo sabamos
Puntaje de Sentimiento: -0.296
Clasificación: neutro

Conversación: Hacemos 15 min caminando
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: Hacemos los labboratotios desde el juanchos
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: Hacemos negocio
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: Hacemos pellas como en el instituto
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: Hacer un mockup tal cual
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: Haces bien menudo laboratorio Jajajajaja
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: Haciendo los labboratotios
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: Haj

Puntaje de Sentimiento: -0.296
Clasificación: neutro

Conversación: No es mucho a lo mejor lo pago
Puntaje de Sentimiento: -0.296
Clasificación: neutro

Conversación: No es que si no iba a morir ah en el fro as que abandone a David e ir con un amigo as que ni tan mal jajaj
Puntaje de Sentimiento: -0.5267
Clasificación: negativo

Conversación: No estoy preparada
Puntaje de Sentimiento: -0.296
Clasificación: neutro

Conversación: No hay ftbol y juanchos al final
Puntaje de Sentimiento: -0.296
Clasificación: neutro

Conversación: No hay posibilidad de hacer un control Z
Puntaje de Sentimiento: -0.296
Clasificación: neutro

Conversación: No hice nada de el laboratorio jaja
Puntaje de Sentimiento: -0.296
Clasificación: neutro

Conversación: No hice nada del curso
Puntaje de Sentimiento: -0.296
Clasificación: neutro

Conversación: No hombre no
Puntaje de Sentimiento: -0.5267
Clasificación: negativo

Conversación: No importa si funciona es suficiente jajajaha
Puntaje de Sentimiento: -0.296
Cl

Conversación: O sea que mi tableau public podis ver como me quedo todo creo que alguna no estaba bien hecha pero
Puntaje de Sentimiento: -0.296
Clasificación: neutro

Conversación: O sea que t tienes que sacar los predictions y eso es el submission
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: O sea si son pequeitos y no tienen a nadie incluso se lo podras intentar vender como proyecto
Puntaje de Sentimiento: -0.296
Clasificación: neutro

Conversación: O solo grficas
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: O verde
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: OG dejndonos deberes para el sbado con cario
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: Octavio lo predijo cul modelo entrenado
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: Ohhh que mona
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: Oigan el sbado vamos a ir a juanchos
Puntaje de Sentimiento: 0.0
Clasificació


Conversación: Que mono por dios
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: Que no decaiga
Puntaje de Sentimiento: -0.296
Clasificación: neutro

Conversación: Que no le ve ni pies ni cabeza y yo normal jajaja
Puntaje de Sentimiento: -0.296
Clasificación: neutro

Conversación: Que no responda ni al mensaje si que es ms raro quiz este de vacaciones en Hawai o algo as
Puntaje de Sentimiento: -0.296
Clasificación: neutro

Conversación: Que no sea nada te echaremos de menos pequea capi
Puntaje de Sentimiento: -0.296
Clasificación: neutro

Conversación: Que no te engae solo est vigilando su presa
Puntaje de Sentimiento: -0.296
Clasificación: neutro

Conversación: Que normalmente est en la escuela por las tardes
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: Que pasa pequeo Capibara
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: Que pereza
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: Que rabia
Puntaje de Sentimiento

Clasificación: neutro

Conversación: Total hasta la cena y ya el sentimiento es de productividad
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: Totalmente
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: Totalmente a mi me pas con lo de regresin y clasificacin
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: Tres dos uno Capi en casa
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: Tu echa cv si ya casi ests graduada
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: Tu no has quedado ni tan mal
Puntaje de Sentimiento: -0.296
Clasificación: neutro

Conversación: Tu que tal ya ests de vuelta
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: Un evento de data y cervezas
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: Un poco bastante dira yo
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: Uy uy
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: Va

Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: adems es recto o me da la impresin de que es rollo cleopatra que por detrs es ms corto todava
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: ahh valee
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: ahh valee ahi se puede ver
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: ahh valee es un ejemplo
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: ahh valee perfecto
Puntaje de Sentimiento: 0.3182
Clasificación: positiva

Conversación: ahh valee que perdida estoy madre mia
Puntaje de Sentimiento: -0.296
Clasificación: neutro

Conversación: ahora te lo puedes probar en filtros de tiktok antes jajajaja
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: al final sacasteis el csv
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: alana
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: alana hoy vienes
Puntaje de Sentimie

Conversación: joer yo tableau no voy a utilizar jajajaja
Puntaje de Sentimiento: -0.296
Clasificación: neutro

Conversación: la cosa es que la veo conectada por eso me parece raro
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: la movida es que no se ni siquiera si los tienen tienen la pagina web y no se si esos datos se los quedara google
Puntaje de Sentimiento: -0.5267
Clasificación: negativo

Conversación: la tarjeta que nos dijo lo damos hoy o lo dejamos para el lunes
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: la voy a dar una semana
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: lo consegui
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: lo de rodri lo probaste
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: lo meti derectamente
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: lo que he visto es que en el test tienen varios Dtype y yo pensaba que habia que igualarlos
Puntaje 


Conversación: tenias muchos atrasados
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: tere como has borrado scipy
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: tere te estoy cotilleando linkeding jeje
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: tere tu eres el naranja
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: tere y con los nulos
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: tere ya te piso los talones
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: tia no me reconoce el sklearn
Puntaje de Sentimiento: 0.2732
Clasificación: positiva

Conversación: tiene mas sentido lo tuyo
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: todos los labboratotios no los ultimos
Puntaje de Sentimiento: -0.296
Clasificación: neutro

Conversación: tu sabes algo 34697319928
Puntaje de Sentimiento: 0.0
Clasificación: neutro

Conversación: tu ya terminaste todos
Puntaje de Sentim